In [1]:
import numpy as np
import math
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry, CircularAperture, CircularAnnulus

In [5]:
def calculate_photometry(flux, aperture_area, annulus_area, std, exptime, flux_flag):
    """
    Calculates photometry values including flux error, instrumental magnitude,
    corrected instrumental magnitude, and their errors.
    
    Parameters:
    - flux: float, the measured light intensity of an astronomical object
    - aperture_area: float, the area of the aperture
    - annulus_area: float, the area of the annulus around the aperture
    - std: float, standard deviation (possibly of the background noise)
    - exptime: float, exposure time
    - flux_flag: any type, additional information about the flux measurement
    
    Returns:
    - phot_table: dictionary, containing the calculated photometry values
    """

    # Calculate the flux error
    flux_error = math.sqrt(flux + (aperture_area * (1 + (math.pi * aperture_area) / (2 * annulus_area)) * (std ** 2)))
    
    # Calculate the instrumental magnitude
    inst_mag = -2.5 * math.log10(flux)
    
    # Correct the instrumental magnitude for exposure time
    corr_inst_mag = inst_mag + (2.5 * math.log10(exptime))
    
    # Calculate the error in the instrumental magnitude
    inst_mag_error = (2.5 * flux_error) / (math.log(10) * flux)
    
    # Calculate the logarithm of the instrumental magnitude error
    log_inst_mag_error = math.log10(inst_mag_error)
    
    # Store all the calculated values in a dictionary
    phot_table = {
        "flux": flux,
        "flux_flag": flux_flag,
        "flux_error": flux_error,
        "inst_mag": inst_mag,
        "corr_inst_mag": corr_inst_mag,
        "inst_mag_error": inst_mag_error,
        "log_inst_mag_error": log_inst_mag_error
    }
    
    # Return the dictionary with the photometry values
    return phot_table


In [6]:
# Load the FITS file
file_name = "/Users/wendymendoza/Desktop/05stacked_77.fits"
image_data = fits.getdata(file_name)

In [7]:
# Define the position of the star (x, y) and radii
x, y = 2043.0361, 2009.0003  # Example coordinates
radius_aperture = 12.45429
radius_inner_annulus = 21.306638
radius_outer_annulus = 30.069002

In [8]:
# Define the apertures
aperture = CircularAperture((x, y), r=radius_aperture)
annulus_aperture = CircularAnnulus((x, y), r_in=radius_inner_annulus, r_out=radius_outer_annulus)
apertures = [aperture, annulus_aperture]

In [9]:
# Get statistics
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)

In [10]:
# Perform aperture photometry
phot_table = aperture_photometry(image_data, apertures)


In [11]:
# Calculate background
bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
bkg_total = bkg_mean * aperture.area

In [12]:
# Subtract background
net_star_flux = phot_table['aperture_sum_0'] - bkg_total

In [13]:
# Calculate magnitude
star_magnitude = -2.5 * np.log10(net_star_flux)

In [15]:
# Call the calculate_photometry function
exptime = 60  # Example exposure time
flux_flag = True  # Example flux flag
photometry_values = calculate_photometry(net_star_flux, aperture.area, annulus_aperture.area, std, exptime, flux_flag)


In [19]:
# Output results
print(f'Star flux: {net_star_flux} e-/s')
print(f'Star magnitude: {star_magnitude}')
print(f'Photometry values: {photometry_values}')



Star flux:   aperture_sum_0 
-----------------
153887.2560243106 e-/s
Star magnitude:    aperture_sum_0  
-------------------
-12.968006639450609
Photometry values: {'flux': <Column name='aperture_sum_0' dtype='float64' length=1>
153887.2560243106, 'flux_flag': True, 'flux_error': 3683.154219485122, 'inst_mag': -12.968006639450609, 'corr_inst_mag': -8.522628513491501, 'inst_mag_error': <Column name='aperture_sum_0' dtype='float64' length=1>
0.02598612768279479, 'log_inst_mag_error': -1.585258431976063}


In [20]:
# Importar librerías
import numpy as np
import math
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry, CircularAperture, CircularAnnulus

In [21]:
def calculate_photometry(flux, aperture_area, annulus_area, std, exptime, flux_flag):
    """
    Calculates photometry values including flux error, instrumental magnitude,
    corrected instrumental magnitude, and their errors.
    
    Parameters:
    - flux: float, the measured light intensity of an astronomical object
    - aperture_area: float, the area of the aperture
    - annulus_area: float, the area of the annulus around the aperture
    - std: float, standard deviation (possibly of the background noise)
    - exptime: float, exposure time
    - flux_flag: any type, additional information about the flux measurement
    
    Returns:
    - phot_table: dictionary, containing the calculated photometry values
    """

    # Calculate the flux error
    flux_error = math.sqrt(flux + (aperture_area * (1 + (math.pi * aperture_area) / (2 * annulus_area)) * (std ** 2)))
    
    # Calculate the instrumental magnitude
    inst_mag = -2.5 * math.log10(flux)
    
    # Correct the instrumental magnitude for exposure time
    corr_inst_mag = inst_mag + (2.5 * math.log10(exptime))
    
    # Calculate the error in the instrumental magnitude
    inst_mag_error = (2.5 * flux_error) / (math.log(10) * flux)
    
    # Calculate the logarithm of the instrumental magnitude error
    log_inst_mag_error = math.log10(inst_mag_error)
    
    # Store all the calculated values in a dictionary
    phot_table = {
        "flux": flux,
        "flux_flag": flux_flag,
        "flux_error": flux_error,
        "inst_mag": inst_mag,
        "corr_inst_mag": corr_inst_mag,
        "inst_mag_error": inst_mag_error,
        "log_inst_mag_error": log_inst_mag_error
    }
    
    # Return the dictionary with the photometry values
    return phot_table


In [22]:
# Cargar el archivo FITS
file_name = "/Users/wendymendoza/Desktop/05stacked_77.fits"
image_data = fits.getdata(file_name)

In [23]:
# Definir la posición de las estrellas [(x1, y1), (x2, y2), ...] y los radios
star_positions = [(2043.0361, 2009.0003), (2050.0, 2010.0)]  # Coordenadas de ejemplo para múltiples estrellas
radius_aperture = 12.45429
radius_inner_annulus = 21.306638
radius_outer_annulus = 30.069002


In [24]:
# Obtener estadísticas
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)


In [26]:
# Realizar fotometría de apertura
phot_table = aperture_photometry(image_data, apertures)

# Calcular fondo
bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
bkg_total = bkg_mean * aperture.area

# Restar fondo
net_star_flux = phot_table['aperture_sum_0'] - bkg_total

# Calcular magnitud
star_magnitude = -2.5 * np.log10(net_star_flux)

# Llamar a la función calculate_photometry
exptime = 60  # Tiempo de exposición de ejemplo
flux_flag = True  # Indicador de flujo de ejemplo
photometry_values = calculate_photometry(net_star_flux, aperture.area, annulus_aperture.area, std, exptime, flux_flag)

# Mostrar resultados
print(f'Flujo de estrella: {net_star_flux} e-/s')
print(f'Magnitud de estrella: {star_magnitude}')
print(f'Valores de fotometría: {photometry_values}')

Flujo de estrella:   aperture_sum_0  
------------------
138425.80043080798 e-/s
Magnitud de estrella:    aperture_sum_0  
-------------------
-12.853042608619184
Valores de fotometría: {'flux': <Column name='aperture_sum_0' dtype='float64' length=1>
138425.80043080798, 'flux_flag': True, 'flux_error': 3681.054678881795, 'inst_mag': -12.853042608619184, 'corr_inst_mag': -8.407664482660074, 'inst_mag_error': <Column name='aperture_sum_0' dtype='float64' length=1>
0.028872177904103936, 'log_inst_mag_error': -1.5395204549118144}
